In [4]:
# %run ./setup_notebook.ipynb
import sys; sys.path.append('/home/pshah/Documents/code/Vape/utils')
print(sys.path)
import glob
import numpy as np
from skimage.external import tifffile
import tifffile as tf
import os
import alloptical_utils_pj as ao
import utils_funcs_pj as pjf

['/home/pshah/Documents/code/Vape/jupyter/Prajay - misc', '/home/pshah/anaconda3/lib/python37.zip', '/home/pshah/anaconda3/lib/python3.7', '/home/pshah/anaconda3/lib/python3.7/lib-dynload', '', '/home/pshah/anaconda3/lib/python3.7/site-packages', '/home/pshah/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/home/pshah/.ipython', '/home/pshah/Documents/code/Vape/utils', '/home/pshah/Documents/code/Vape/', '/home/pshah/Documents/code/Vape/utils']


## convert single tiff frames into tiff stack

In [2]:
tiff_paths = '/Volumes/Extreme SSD/oxford-data/2020-03-06/t06/*.tif'
ao.make_tiff_stack(tiff_paths, save_as='/Volumes/Extreme SSD/oxford-data/2020-03-06/t06.tif')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## remove artifact from alloptical experiments

In [5]:
## DOESN't WORK INDEPENDENTLY OF THE EXPOBJECT CURRENTLY

### make a new tiff file (not for suite2p) with the first photostim frame whitened, and save new tiff
tiff_path = '/home/pshah/mnt/qnap/Data/2020-03-04/2020-03-04_t-014/2020-03-04_t-014_Cycle00001_Ch3.tif'

im_stack = tf.imread(tiff_path, key=range(exp_obj.n_frames))


frames_to_whiten = []
for j in exp_obj.stim_start_frames[0]:
        frames_to_whiten.append(j)

im_stack_1 = im_stack
a = np.full_like(im_stack_1[0], fill_value=0)
a[0:100,0:100] = 5000.
for frame in frames_to_whiten:
    im_stack_1[frame-3:frame-1] = a

frames_to_remove = []
for j in exp_obj.stim_start_frames[0]:
    for i in range(0, exp_obj.duration_frames+1): # usually need to remove 1 more frame than the stim duration, as the stim isn't perfectly aligned with the start of the imaging frame
        frames_to_remove.append(j+i)

im_stack_1 = np.delete(im_stack, frames_to_remove, axis=0)

tf.imwrite("/home/pshah/mnt/qnap/Data/2020-03-04/2020-03-04_t-014/2020-03-04_t014_artifactrem.tif",
           im_stack_1, photometric='minisblack')

# find the frames which are now photostim frames (relative to the new length after frames removed)
new_tiff = "/home/pshah/mnt/qnap/Data/2020-03-04/2020-03-04_t-014/2020-03-04_t014_artifactrem.tif"
new_stack = tf.imread(new_tiff, key=(range(len(im_stack_1))))

new_photostim_frames = []
for i in range(len(new_stack)):
    if new_stack[i][0][0]==1001:
        new_photostim_frames.append(i)

NameError: name 'exp_obj' is not defined